Import modules

In [1]:
import boto3
import json
import os
from dotenv import load_dotenv 
import cv2


Get environment variables and create AWS Session

In [2]:

dotenv_path = os.path.join("..", 'interface', '.env')
load_dotenv(dotenv_path)

#Connect to AWS Ressource, Credentials are optionals as we are accessing from the same AWS user
client=boto3.client('rekognition',aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME"))
s3 = boto3.client('s3',
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("REGION_NAME")
)

#Define bucket name to access
bucket_name = os.getenv("BUCKET_NAME")

#Folder of real classroom photos
classroomFolder=r"C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos"

Take all images of the folder and for each draw detected faces and save it to FacesDetectedInClassroom

In [3]:

#Cycle that only draws rectangles where there are faces according to Rekognition's algorithm
i=1
boundingBoxDetectedFaces=[]

for file in os.listdir(classroomFolder):
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos\{file}' 
    #convert files to bytes in order to be able to read it

    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    # Read the image with OpenCV
    image = cv2.imread(camera_local_file)
    image_height, image_width, _ = image.shape

    boundingBoxDetectedFacesithImage=[]

    # Draw bounding boxes for each detected face
    for face in responseDetection['FaceDetails']:
        bbox = face['BoundingBox']
        left = int(bbox['Left'] * image_width)
        top = int(bbox['Top'] * image_height)
        width = int(bbox['Width'] * image_width)
        height = int(bbox['Height'] * image_height)
        cv2.rectangle(
            image, 
            (left, top), 
            (left + width, top + height), 
            (37, 173, 250), 
            2  
        )
        boundingBoxDetectedFacesithImage.append((left, top, width,height))
    #Each array element represents a sub-array of tuples containing the bonding boxes    
    boundingBoxDetectedFaces.append(boundingBoxDetectedFacesithImage) 
    # Print number of faces detected
    print(f"{len(responseDetection['FaceDetails'])} Faces were detected")

    # Save the annotated image
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\FacesDetectedInClassroom\faces_detected_{i}.jpeg"
    cv2.imwrite(output_path, image)
    #Increase the photo's index
    i=i+1


21 Faces were detected
21 Faces were detected
9 Faces were detected
9 Faces were detected
9 Faces were detected
18 Faces were detected
19 Faces were detected


In [4]:

def calculate_iou_detection(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    x1, y1, w1, h1 = box1
    x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou




In [39]:
true=True
groundTruth=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":18,"y":509,"width":31,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":166,"y":487,"width":30,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":36,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":345,"y":485,"width":36,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":371,"y":425,"width":30,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":410,"y":583,"width":50,"height":77},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":570,"y":543,"width":51,"height":65},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":459,"width":46,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":29,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1159,"y":367,"width":24,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1192,"y":332,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":909,"y":349,"width":23,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":365,"width":26,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":737,"y":403,"width":27,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":715,"y":351,"width":22,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":621,"y":377,"width":26,"height":35},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":439,"y":369,"width":23,"height":26},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":611,"y":338,"width":21,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":694,"y":331,"width":18,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":278,"width":19,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":544,"y":428,"width":39,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":26,"y":506,"width":31,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":158,"y":485,"width":32,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":174,"y":537,"width":38,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":425,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":341,"y":481,"width":37,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":442,"y":367,"width":23,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":427,"y":579,"width":53,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":582,"y":546,"width":52,"height":59},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":458,"width":47,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1036,"y":394,"width":28,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":379,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1184,"y":329,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":910,"y":349,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":364,"width":27,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":730,"y":397,"width":28,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":719,"y":352,"width":21,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":619,"y":379,"width":27,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":615,"y":338,"width":22,"height":23},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":694,"y":335,"width":18,"height":17},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":429,"width":38,"height":43},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":660,"y":277,"width":20,"height":22},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":67,"y":800,"width":68,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":357,"y":718,"width":64,"height":68},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":531,"y":593,"width":54,"height":61},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":603,"y":499,"width":38,"height":46},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":454,"width":30,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":773,"y":542,"width":39,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1024,"y":483,"width":32,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":433,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":411,"width":25,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":72,"y":800,"width":65,"height":74},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":362,"y":720,"width":62,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":541,"width":42,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":551,"y":595,"width":48,"height":56},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":501,"width":36,"height":47},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":26,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1024,"y":481,"width":32,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":433,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":451,"width":28,"height":39},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":69,"y":799,"width":67,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":366,"y":722,"width":59,"height":65},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":610,"y":499,"width":33,"height":48},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":549,"y":589,"width":50,"height":62},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":796,"y":454,"width":26,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":539,"width":42,"height":49},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":481,"width":34,"height":33},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":411,"width":26,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":61,"y":844,"width":66,"height":75},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":216,"y":721,"width":64,"height":71},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":429,"y":700,"width":56,"height":61},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":585,"y":653,"width":46,"height":54},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":611,"width":44,"height":51},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":973,"y":534,"width":33,"height":40},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1076,"y":514,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":956,"y":469,"width":25,"height":32},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":500,"width":21,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":526,"width":23,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":701,"y":531,"width":36,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":566,"width":39,"height":50},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":154,"y":576,"width":34,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":192,"y":550,"width":24,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":289,"y":518,"width":25,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":241,"y":579,"width":29,"height":34},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":556,"width":28,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":679,"y":500,"width":21,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":45,"y":848,"width":65,"height":76},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":725,"width":59,"height":70},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":699,"width":56,"height":60},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":657,"width":46,"height":53},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":43,"height":55},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":970,"y":539,"width":31,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":520,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":955,"y":473,"width":23,"height":30},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":900,"y":501,"width":24,"height":29},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":808,"y":527,"width":23,"height":28},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":707,"y":530,"width":30,"height":36},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":576,"y":570,"width":36,"height":45},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":131,"y":573,"width":33,"height":38},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":185,"y":554,"width":25,"height":27},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":291,"y":503,"width":23,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":245,"y":578,"width":29,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":556,"width":25,"height":31},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":604,"y":437,"width":14,"height":18},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":673,"y":489,"width":16,"height":24},"region_attributes":{"name":"not_defined","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in groundTruth:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredgroundTruthObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredgroundTruthObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        tuplesnthPhoto.append((region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1

In [40]:
i=0

for element in tuples:
    TP=0
    TN=0
    FN=0
    FP=0
    verifiedPredictions=0    
    print(f"------Results for image {i+1}------")
    for face in element: #Compare each face of ground truth with the predictions' array. If found, add TP, if not identified, add FN
        found=False
        for detectedFace in boundingBoxDetectedFaces[i]:
            IoU=calculate_iou_detection(face, detectedFace)
            if (IoU>=0.5):
                found=True
                TP=TP+1
                verifiedPredictions=verifiedPredictions+1
                break
        if (found==False):
            FN=FN+1
    FP=FP+(len(boundingBoxDetectedFaces[i])-verifiedPredictions)
    print(F"TP: {TP}, TN:{TN}, FP:{FP} FN:{FN}")



    i=i+1
        


------Results for image 1------
TP: 21, TN:0, FP:0 FN:0
------Results for image 2------
TP: 21, TN:0, FP:0 FN:0
------Results for image 3------
TP: 9, TN:0, FP:0 FN:0
------Results for image 4------
TP: 9, TN:0, FP:0 FN:0
------Results for image 5------
TP: 9, TN:0, FP:0 FN:0
------Results for image 6------
TP: 18, TN:0, FP:0 FN:0
------Results for image 7------
TP: 19, TN:0, FP:0 FN:0


Take all images of the folder and all student's personal image and draw a unique color rectangle if recognized

In [ ]:
colors = [
    (255, 0, 0),      # Azul puro
    (0, 255, 0),      # Verde puro
    (0, 0, 255),      # Rojo puro
    (255, 255, 0),    # Cian
    (255, 0, 255),    # Magenta
    (0, 255, 255),    # Amarillo
    (128, 0, 0),      # Azul oscuro
    (0, 128, 0),      # Verde oscuro
    (0, 0, 128),      # Rojo oscuro
    (128, 128, 0),    # Verde azulado
    (128, 0, 128),    # Púrpura
    (0, 128, 128),    # Marrón
    (64, 0, 128),     # Rojo violeta
    (128, 128, 255),  # Rosa pálido
    (0, 255, 128),    # Verde claro
    (255, 128, 0),    # Azul claro
    (128, 255, 128),  # Verde menta
    (192, 64, 64),    # Azul grisáceo
    (64, 192, 192),   # Turquesa
    (255, 128, 192),  # Violeta claro
    (32, 64, 255)     # Naranja rojizo
]
#Array that contains all faces recognized in all test images. Each photo corresponds to the n-th array element
boundingBoxRecognizedFaces=[]
recognized_counter_array=[]
m=1
photo_index=1

#Create image of the class with all faces recognized in it
for file in os.listdir(classroomFolder):
    camera_local_file = rf'C:\Users\User\Desktop\FinalProject\ImagesRekognition\RealClassroomPhotos\{file}' 
    print(f"Using index {photo_index}")

    if photo_index>=1 and photo_index<=2:
        selectedCourse="StudentsRobotics"
        student_array=["AlbaQuiroz", "AlejandroCasallas","AlejandroVelasco","AlexanderDominguez","AndresAnillo","CarlosBornacelly","DavidChaparro"
                    ,"EstebanToro","GabrielCastanez","GiohanOlivares","JesusCotes","JoseVilla", "ManuelRangel","MariaBerdugo","MariaLopez",
                    "MauricioDeLaHoz", "SergioFonseca","YordiRochel"]    
    elif photo_index>=3 and photo_index<=5:
        selectedCourse="StudentsElectronicDesign1"
        student_array=["AllysonSalom", "AngieValencia","DanielDelgado","DiegoGomez", "FelipeOspino","HabidAbdala","WilmanDaza"]

    elif photo_index>=6 and photo_index<=7:
        selectedCourse="StudentsElectronicDesign2"
        student_array=["AliRada","AndreaQuintero","AndresFabregas","AndresNarvaez","BrayanRubiano","DiegoGomez","EdwinUtria","GabrielaBecerra",
                    "JuanHernandez","JuanQuintero","JuanSanchez","LucianaDeLaRosa","MariaFerrer","MayraCarreno",
                    "OctavioMorales","SamirBarcelo","SteevenBallena"]

    #convert files to bytes in order to be able to read it
    with open(camera_local_file, 'rb') as image_file:
        camera_file_bytes = image_file.read()
    responseDetection=client.detect_faces( Image={'Bytes': camera_file_bytes})
    image = cv2.imread(camera_local_file)

    k=0
    recognized_counter=0

    boundingBoxRecognizedFacesithImage=[]

    for student_name in student_array:
        student_found = False  # Bandera para controlar si el estudiante fue encontrado
        
        student_id_photo=rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\{selectedCourse}"+ rf"\{student_name}.jpeg"
        with open(student_id_photo, 'rb') as image_file:
            student_id_photo_bytes = image_file.read()

        #Look for the face in SourceImage evaluating all the faces in TargetImage
        responseRecognition=client.compare_faces(
            SourceImage={'Bytes': student_id_photo_bytes},
            TargetImage={'Bytes': camera_file_bytes},
            SimilarityThreshold=80
        )
        # Read the target image with OpenCV

        # Get image dimensions
        image_height, image_width, _ = image.shape

        # Draw bounding boxes for matched faces
        for faceMatch in responseRecognition['FaceMatches']:
            # Get bounding box coordinates
            bbox = faceMatch['Face']['BoundingBox']
            
            # Convert normalized coordinates to pixel coordinates
            left = int(bbox['Left'] * image_width)
            top = int(bbox['Top'] * image_height)
            width = int(bbox['Width'] * image_width)
            height = int(bbox['Height'] * image_height)
        
            # Draw green rectangle (to differentiate from previous red detection boxes)
            cv2.rectangle(
                image, 
                (left, top), 
                (left + width, top + height), 
                colors[k],  # Green color in BGR
                2,
            )
            
            padding = 5  # Espacio entre el rectángulo y el cuadro de texto
            text_box_height = 25  # Altura del cuadro de texto
            # Agregar texto dentro del recuadro de texto
            cv2.putText(image, f"{student_name}", (left + 10, top - padding - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, colors[k], 1)

            recognized_counter=recognized_counter+1
            print('Face at coordinates ' + str(bbox) + ' matches with ' + str(faceMatch['Similarity']) + '% confidence'+ f"and is likely {student_name}")

            #Save the identity and region as a tuple in the n-th array element that corresponds to the n-th image
            boundingBoxRecognizedFacesithImage.append((student_name,left,top,width,height))
            student_found = True  # Marcamos que este estudiante fue encontrado
        
        # Si no se encontró al estudiante, agregar una tupla con valores cero
        if not student_found:
            boundingBoxRecognizedFacesithImage.append((student_name, 0, 0, 0, 0))
            print(f"Student {student_name} not found in the image")
            
        k=k-1
    boundingBoxRecognizedFaces.append(boundingBoxRecognizedFacesithImage)
    recognized_counter_array.append(recognized_counter)
    # Save the annotated image
    output_path = rf"C:\Users\User\Desktop\FinalProject\ImagesRekognition\StudentFoundInClassroom\faces_in_classroom_{m}_{recognized_counter}.jpeg"
    cv2.imwrite(output_path, image)
    print(f"Annotated image saved to {output_path}")
    m=m+1
    photo_index=photo_index+1

Obtain tuples from base line (Manual correction and verification)

In [49]:
true=True
faceGallery=[
{"classroom1.jpeg156424":{"filename":"classroom1.jpeg","size":156424,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":348,"width":21,"height":27},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":733,"y":402,"width":29,"height":34},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":344,"y":485,"width":38,"height":44},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":612,"y":337,"width":20,"height":23},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":164,"y":481,"width":35,"height":38},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":567,"y":548,"width":57,"height":58},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":403,"y":582,"width":59,"height":78},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":332,"width":20,"height":22},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":15,"y":506,"width":35,"height":40},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1189,"y":330,"width":31,"height":31},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":122,"y":538,"width":38,"height":55},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":440,"y":368,"width":24,"height":26},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":806,"y":460,"width":47,"height":55},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1158,"y":366,"width":26,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":399,"width":30,"height":37},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":717,"y":350,"width":22,"height":27},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":547,"y":431,"width":37,"height":44},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":372,"y":427,"width":28,"height":33},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom2.jpeg152579":{"filename":"classroom2.jpeg","size":152579,"regions":[{"shape_attributes":{"name":"rect","x":910,"y":347,"width":25,"height":28},"region_attributes":{"name":"AlbaQuiroz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":729,"y":394,"width":31,"height":35},"region_attributes":{"name":"AlejandroCasallas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":340,"y":481,"width":37,"height":46},"region_attributes":{"name":"AlejandroVelasco","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":617,"y":336,"width":20,"height":25},"region_attributes":{"name":"AlexanderDominguez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":160,"y":481,"width":31,"height":37},"region_attributes":{"name":"AndresAnillo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":580,"y":541,"width":55,"height":65},"region_attributes":{"name":"CarlosBornacelly","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":428,"y":580,"width":51,"height":76},"region_attributes":{"name":"DavidChaparro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":693,"y":333,"width":20,"height":20},"region_attributes":{"name":"EstebanToro","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":28,"y":504,"width":29,"height":38},"region_attributes":{"name":"GabrielCastanez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1185,"y":328,"width":27,"height":32},"region_attributes":{"name":"GiohanOlivares","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":171,"y":538,"width":40,"height":57},"region_attributes":{"name":"JesusCotes","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":443,"y":366,"width":23,"height":27},"region_attributes":{"name":"JoseVilla","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":809,"y":459,"width":46,"height":53},"region_attributes":{"name":"ManuelRangel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1126,"y":375,"width":25,"height":33},"region_attributes":{"name":"MariaBerdugo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1039,"y":395,"width":25,"height":36},"region_attributes":{"name":"MariaLopez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":720,"y":350,"width":20,"height":30},"region_attributes":{"name":"MauricioDeLaHoz","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":542,"y":428,"width":38,"height":43},"region_attributes":{"name":"SergioFonseca","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":373,"y":425,"width":27,"height":32},"region_attributes":{"name":"YordiRochel","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom3.jpg155001":{"filename":"classroom3.jpg","size":155001,"regions":[{"shape_attributes":{"name":"rect","x":526,"y":587,"width":59,"height":70},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":605,"y":499,"width":37,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":537,"width":43,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":31,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":408,"width":23,"height":32},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1107,"y":430,"width":26,"height":32},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1026,"y":479,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom4.jpg156291":{"filename":"classroom4.jpg","size":156291,"regions":[{"shape_attributes":{"name":"rect","x":552,"y":592,"width":48,"height":62},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":609,"y":500,"width":35,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":770,"y":538,"width":39,"height":51},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":453,"width":28,"height":39},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1023,"y":480,"width":31,"height":37},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":25,"height":33},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1155,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom5.jpg156524":{"filename":"classroom5.jpg","size":156524,"regions":[{"shape_attributes":{"name":"rect","x":551,"y":592,"width":49,"height":59},"region_attributes":{"name":"DanielDelgado","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":769,"y":540,"width":42,"height":49},"region_attributes":{"name":"WilmanDaza","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":607,"y":500,"width":36,"height":45},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":795,"y":452,"width":29,"height":41},"region_attributes":{"name":"FelipeOspino","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1020,"y":480,"width":33,"height":39},"region_attributes":{"name":"AllysonSalom","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1108,"y":431,"width":26,"height":34},"region_attributes":{"name":"AngieValencia","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1156,"y":410,"width":25,"height":30},"region_attributes":{"name":"HabidAbdala","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom6.jpg153504":{"filename":"classroom6.jpg","size":153504,"regions":[{"shape_attributes":{"name":"rect","x":59,"y":845,"width":67,"height":74},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":221,"y":721,"width":58,"height":72},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":434,"y":701,"width":50,"height":60},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":584,"y":657,"width":47,"height":47},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":772,"y":616,"width":42,"height":45},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":976,"y":539,"width":29,"height":37},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1077,"y":516,"width":26,"height":30},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":957,"y":472,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":903,"y":501,"width":23,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":811,"y":526,"width":21,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":704,"y":532,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":574,"y":567,"width":40,"height":49},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":29},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":243,"y":581,"width":30,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":520,"width":21,"height":26},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":193,"y":553,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":155,"y":576,"width":31,"height":36},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
,{"classroom7.jpg155011":{"filename":"classroom7.jpg","size":155011,"regions":[{"shape_attributes":{"name":"rect","x":46,"y":849,"width":64,"height":79},"region_attributes":{"name":"MayraCarreno","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":219,"y":724,"width":58,"height":71},"region_attributes":{"name":"JuanSanchez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":430,"y":696,"width":55,"height":65},"region_attributes":{"name":"JuanQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":589,"y":661,"width":46,"height":49},"region_attributes":{"name":"SamirBarcelo","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":771,"y":612,"width":44,"height":54},"region_attributes":{"name":"EdwinUtria","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":969,"y":539,"width":32,"height":39},"region_attributes":{"name":"OctavioMorales","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":1069,"y":518,"width":29,"height":33},"region_attributes":{"name":"MariaFerrer","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":954,"y":475,"width":25,"height":28},"region_attributes":{"name":"SteevenBallena","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":898,"y":501,"width":27,"height":29},"region_attributes":{"name":"BrayanRubiano","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":807,"y":527,"width":24,"height":29},"region_attributes":{"name":"AndreaQuintero","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":575,"y":569,"width":38,"height":47},"region_attributes":{"name":"DiegoGomez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":706,"y":531,"width":31,"height":34},"region_attributes":{"name":"JuanHernandez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":343,"y":557,"width":27,"height":31},"region_attributes":{"name":"AndresFabregas","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":290,"y":505,"width":24,"height":29},"region_attributes":{"name":"AliRada","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":244,"y":579,"width":28,"height":31},"region_attributes":{"name":"GabrielaBecerra","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":186,"y":555,"width":24,"height":26},"region_attributes":{"name":"LucianaDeLaRosa","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}},{"shape_attributes":{"name":"rect","x":134,"y":574,"width":31,"height":38},"region_attributes":{"name":"AndresNarvaez","type":"unknown","image_quality":{"good":true,"frontal":true,"good_illumination":true}}}],"file_attributes":{"caption":"","public_domain":"no","image_url":""}}}
]

m=1
#The N-Element contains tuples of the N-Th image with informations of students to be recognized
tuples=[]
for element in faceGallery:
    i=1
    #(f"photo {m}-----------------------------------------")
    filteredFaceGalleryObject=element[next(iter(element))]["regions"]
    tuplesnthPhoto=[]
    for element in filteredFaceGalleryObject:
        #print(f"Person {i}-----")
        region=element["shape_attributes"]
        identity=element["region_attributes"]
        #print(f"x = {region["x"]} \ny = {region["y"]} \nwidth = {region["width"]}\nheight = {region["height"]} \nidentifier = {identity["name"]}")
        tuplesnthPhoto.append((identity["name"],region["x"],region["y"], region["width"], region["height"]))
        i=i+1
    tuples.append(tuplesnthPhoto)
    m=m+1


Code to calculate IoU

In [22]:

def calculate_iou(box1, box2):
    """
    Calculate IoU between 2 boxes.
    box1, box2: tuples like (x, y, width, height)
    return IoU value that ranges from 0 to 1
    """
    # Extraer coordenadas
    name1,x1, y1, w1, h1 = box1
    name2, x2, y2, w2, h2 = box2
    
    # Calcular coordenadas de la intersección
    x_intersect_1 = max(x1, x2)
    y_intersect_1 = max(y1, y2)
    x_intersect_2 = min(x1 + w1, x2 + w2)
    y_intersect_2 = min(y1 + h1, y2 + h2)
    
    # Comprobar si hay intersección
    if x_intersect_2 <= x_intersect_1 or y_intersect_2 <= y_intersect_1:
        return 0.0
    
    # Calcular áreas
    area_intersect = (x_intersect_2 - x_intersect_1) * (y_intersect_2 - y_intersect_1)
    area_box1 = w1 * h1
    area_box2 = w2 * h2
    
    # Calcular unión
    area_union = area_box1 + area_box2 - area_intersect
    
    # Calcular IoU
    iou = area_intersect / area_union
    
    return iou


Iterate through the base line and compare it to the results obtained by Rekognition. Then calculate TP, TN, FP, FN

In [50]:
#It searchs based on face gallery/Base Line: It compares my definition of the existance of specific students and compares it to the found studens
#The result wouldn't be affected if the search direction is reversed

i=0
#Loop through all the photos
for photoObject in tuples:
    print(f"Results for photo {i+1}")
    TP=0
    TN=0
    FP=0
    FN=0
    #Obtain the array of faces recognized with Rekognition in that photo
    tuplesToCompare=boundingBoxRecognizedFaces[i]

    listOfNamesBaseline=[]
    for element in photoObject:
        listOfNamesBaseline.append(element[0])

    listOfNamesModel=[]
    for element in tuplesToCompare:
        listOfNamesModel.append(element[0])


    for element in photoObject:
        if(element[0] in listOfNamesModel):
            for identity in tuplesToCompare:
                IoU=calculate_iou(element,identity)

                #Case True Positive: Identity and bounding box match
                if(IoU>=0.5 and element[0]==identity[0]):
                    TP=TP+1

                #Case False Positive: no identity match, however, the bounding boxes are similar
                if(IoU>=0.5 and element[0]!=identity[0]):
                    FP=FP+1

                #Case True Negative: No bounding box match. However, the identity is different
                if(IoU<0.5 and element[0]!=identity[0]):
                    TN=TN+1
                #Case False Negative: The identity matches. However, bonding boxes do not correspond
                if(IoU<0.5 and element[0]==identity[0]):
                    FN=FN+1 
            
            #Correction to take into account if the model doesn't mistake identities of strangers: Detected Faces-Faces identified        
            TN=TN+len(boundingBoxDetectedFaces[i])-len(listOfNamesModel)
        else:
            FN=FN+1
            TN=TN+len(boundingBoxDetectedFaces[i])-1
            print(f"{element[0]} not in model's list and adds 1 FN and {len(boundingBoxDetectedFaces)-1} TN")

    #print(recognized_counter_array[i])
    print(f"TP : {TP} \nTN : {TN} \nFP : {FP} \nFN : {FN}")
    i=i+1

#Buscar metrica deteccion

Results for photo 1
TP : 17 
TN : 360 
FP : 0 
FN : 1
Results for photo 2
TP : 14 
TN : 360 
FP : 0 
FN : 4
Results for photo 3
TP : 6 
TN : 56 
FP : 0 
FN : 1
Results for photo 4
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 5
TP : 7 
TN : 56 
FP : 0 
FN : 0
Results for photo 6
TP : 14 
TN : 289 
FP : 0 
FN : 3
Results for photo 7
TP : 14 
TN : 306 
FP : 0 
FN : 3


In [25]:
listOfNamesModel

['AllysonSalom',
 'AngieValencia',
 'DanielDelgado',
 'DiegoGomez',
 'FelipeOspino',
 'HabidAbdala',
 'WilmanDaza']